In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.impute import SimpleImputer
from plotly.subplots import make_subplots
!pip install feature_engine
from feature_engine.encoding import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 276 kB 26.7 MB/s 
     |████████████████████████████████| 9.8 MB 45.4 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [2]:
# To Import Data from google drive (authentication needed)
from google.colab import drive 
drive.mount('/content/gdrive')

import pandas as pd 
Data=pd.read_excel('gdrive/My Drive/Personal Data Science Projects/Flight Tickets Price Prediction/Data_Train.xlsx')

Test_Data = pd.read_excel('gdrive/My Drive/Personal Data Science Projects/Flight Tickets Price Prediction/Test_set.xlsx')

Mounted at /content/gdrive


In [3]:
Data.head(10)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302
5,SpiceJet,24/06/2019,Kolkata,Banglore,CCU → BLR,09:00,11:25,2h 25m,non-stop,No info,3873
6,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,18:55,10:25 13 Mar,15h 30m,1 stop,In-flight meal not included,11087
7,Jet Airways,01/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:00,05:05 02 Mar,21h 5m,1 stop,No info,22270
8,Jet Airways,12/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:55,10:25 13 Mar,25h 30m,1 stop,In-flight meal not included,11087
9,Multiple carriers,27/05/2019,Delhi,Cochin,DEL → BOM → COK,11:25,19:15,7h 50m,1 stop,No info,8625


In [4]:
Data.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

In [5]:
Test_Data.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
dtype: int64

In [6]:
# Row having missing values
Data[ Data['Route'].isnull() ]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


In [7]:
Data[ (Data['Source']=='Delhi') & (Data['Destination']=='Cochin') & (Data['Dep_Time']=='09:45') & (Data['Duration']=="23h 40m") & (Data['Price']==7480) ]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
1616,Air India,24/05/2019,Delhi,Cochin,DEL → MAA → COK,09:45,09:25 25 May,23h 40m,1 stop,No info,7480
2554,Air India,15/05/2019,Delhi,Cochin,DEL → MAA → COK,09:45,09:25 16 May,23h 40m,1 stop,No info,7480
4351,Air India,27/06/2019,Delhi,Cochin,DEL → MAA → COK,09:45,09:25 28 Jun,23h 40m,1 stop,No info,7480
7896,Air India,12/06/2019,Delhi,Cochin,DEL → MAA → COK,09:45,09:25 13 Jun,23h 40m,1 stop,No info,7480
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


In [8]:
Data.at[9039,'Route'] = 'DEL → MAA → COK'
Data.at[9039,'Total_Stops'] = '1 stop'

In [9]:
Data.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Price              0
dtype: int64

In [10]:
target_df = Data[['Price']]
Data.drop(columns='Price',inplace=True)

In [11]:
categorical_features_name = ['Airline','Source','Destination','Total_Stops','Additional_Info']
date_time_features_name = [ "Date_of_Journey", 'Dep_Time', 'Arrival_Time', 'Duration' ]
diff_categorical_feature = ['Route']
numerical_features_name = list( set(Data.columns.values)-set(categorical_features_name)-set(date_time_features_name)-set(diff_categorical_feature) )

In [12]:
i=0
X_num_train = Data[categorical_features_name]
fig = make_subplots( rows=1, cols = X_num_train.shape[1],subplot_titles = X_num_train.columns.values )
for feature in X_num_train.columns.values:
  fig.add_trace( go.Histogram( x=X_num_train[feature], name=feature), row=1,col=i+1 )
  i = i+1

fig.update_layout( width=5000 )
fig.show()

In [13]:
for column in categorical_features_name:
  print( "Feature:- ", column )
  print("Unique Values:-", list( Data[column].unique() ) )
  print("--------------------------------------------------------------")
  print("\n")

Feature:-  Airline
Unique Values:- ['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet', 'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia', 'Vistara Premium economy', 'Jet Airways Business', 'Multiple carriers Premium economy', 'Trujet']
--------------------------------------------------------------


Feature:-  Source
Unique Values:- ['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai']
--------------------------------------------------------------


Feature:-  Destination
Unique Values:- ['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad']
--------------------------------------------------------------


Feature:-  Total_Stops
Unique Values:- ['non-stop', '2 stops', '1 stop', '3 stops', '4 stops']
--------------------------------------------------------------


Feature:-  Additional_Info
Unique Values:- ['No info', 'In-flight meal not included', 'No check-in baggage included', '1 Short layover', 'No Info', '1 Long layover', 'Change airports', 'Business class', 'Red-eye 

#### Renaming values having same meaning

In [14]:
def preprocessing_renaming(df):
  df['Additional_Info'].replace(to_replace='No info',value='No Info',inplace=True)
  df['Destination'].replace(to_replace='New Delhi',value='Delhi',inplace=True)
  return df

In [15]:
Data = preprocessing_renaming(Data)
Test_Data = preprocessing_renaming(Test_Data)

#### One Hot Encoding of Categorical Variables

In [16]:
def onehotencoder(Data):
  ohe = OneHotEncoder( drop_last=True, variables=['Source','Destination','Additional_Info', 'Airline'] )
  ohe.fit( Data )
  Data = ohe.transform(Data)
  return Data, ohe

In [17]:
Data, ohe = onehotencoder(Data)
Test_Data = ohe.transform(Test_Data)

In [18]:
ohe.encoder_dict_

{'Additional_Info': ['No Info',
  'In-flight meal not included',
  'No check-in baggage included',
  '1 Short layover',
  '1 Long layover',
  'Change airports',
  'Business class',
  'Red-eye flight'],
 'Airline': ['IndiGo',
  'Air India',
  'Jet Airways',
  'SpiceJet',
  'Multiple carriers',
  'GoAir',
  'Vistara',
  'Air Asia',
  'Vistara Premium economy',
  'Jet Airways Business',
  'Multiple carriers Premium economy'],
 'Destination': ['Delhi', 'Banglore', 'Cochin', 'Kolkata'],
 'Source': ['Banglore', 'Kolkata', 'Delhi', 'Chennai']}

In [19]:
Data.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', '4 stops'],
      dtype=object)

#### Ordinal Encoding

In [20]:
def ordinal_encoder(df):
  ord_end = OrdinalEncoder( categories=[['non-stop','1 stop','2 stops','3 stops','4 stops']] )
  feature_df = df[['Total_Stops']] 
  ord_end.fit(feature_df)
  df['Total_Stops'] = ord_end.transform(feature_df)
  df['Total_Stops'] = df['Total_Stops'].astype(int)
  return df, ord_end

In [21]:
Data, ord_end =  ordinal_encoder(Data)
Test_Data['Total_Stops'] = ord_end.transform(Test_Data[['Total_Stops']])

In [22]:
def preprocessing_Arrival_Time(Data):
  temp_df = Data['Arrival_Time'].str.split(" ",expand=True)
  temp_df.drop(columns=[1,2],inplace=True) # Dropping the days and month columns becuase it contains many missing values (around 6k)
  temp_df = temp_df[0].str.split(':',expand=True)
  temp_df.rename( columns={0:'hour',1:'minutes'}, inplace=True )
  temp_df = temp_df.astype(int)
  return temp_df['hour']*60 + temp_df['minutes']

In [23]:
Data['Arrival_Time'] = preprocessing_Arrival_Time(Data)
Test_Data['Arrival_Time'] = preprocessing_Arrival_Time(Test_Data)

In [24]:
def preprocessing_Dep_Time(Data):
  temp_df = Data['Dep_Time'].str.split(":",expand=True)
  temp_df.rename( columns={0:'hour',1:'minutes'}, inplace=True )
  temp_df = temp_df.astype(int)
  return temp_df['hour']*60 + temp_df['minutes']

In [25]:
Data['Dep_Time'] = preprocessing_Dep_Time(Data)
Test_Data['Dep_Time'] = preprocessing_Dep_Time(Test_Data)

In [26]:
def preprocessing_Date_of_Journey(Data):
  temp_df = Data['Date_of_Journey'].str.split("/",expand=True)
  temp_df.rename( columns={0:'Journey_day',1:'Journey_month',2:'Journey_year'},inplace=True )
  Data.drop(columns='Date_of_Journey', inplace=True)
  Data = pd.concat( [Data,temp_df.astype(int)], axis=1 )
  return Data

In [27]:
Data = preprocessing_Date_of_Journey(Data)
Test_Data = preprocessing_Date_of_Journey(Test_Data)

In [28]:
temp_df = Data[['Duration']]
temp_df = temp_df['Duration'].str.split(" ",expand=True)
temp_df.rename(columns={0:'hours',1:'minutes'},inplace=True)

temp_df.at[6474,'hours'] = '0h'
temp_df.at[6474,'minutes'] = '5m'  # temp_df[temp_df['hours']=='5m']

# print( temp_df['minutes'].unique() )
SI = SimpleImputer(missing_values=None,fill_value='0m',strategy='constant')
SI.fit( temp_df['minutes'].values.reshape(-1,1) )
temp_df['minutes'] = SI.transform( temp_df['minutes'].values.reshape(-1,1) )

for i in range(temp_df.shape[0]):
  temp_df.at[i,'minutes'] = int( temp_df['minutes'][i][:-1] )
  temp_df.at[i,'hours'] = int( temp_df['hours'][i][:-1] )

Data['Duration'] = ( temp_df['minutes'] + 60*temp_df['hours'] ).astype(int)

In [29]:
temp_df = Test_Data[['Duration']]
temp_df = temp_df['Duration'].str.split(" ",expand=True)
temp_df.rename(columns={0:'hours',1:'minutes'},inplace=True)

temp_df.at[2660,'hours'] = '0h'
temp_df.at[2660,'minutes'] = '5m'  # temp_df[temp_df['hours']=='5m']

# print( temp_df['minutes'].unique() )
SI = SimpleImputer(missing_values=None,fill_value='0m',strategy='constant')
SI.fit( temp_df['minutes'].values.reshape(-1,1) )
temp_df['minutes'] = SI.transform( temp_df['minutes'].values.reshape(-1,1) )

for i in range(temp_df.shape[0]):
  temp_df.at[i,'minutes'] = int( temp_df['minutes'][i][:-1] )
  temp_df.at[i,'hours'] = int( temp_df['hours'][i][:-1] )

Test_Data['Duration'] = ( temp_df['minutes'] + 60*temp_df['hours'] ).astype(int)

In [30]:
list_sentences = []
for id in Data.index.values:
    sentence = Data.at[id,'Route']
    list_sentences.append( sentence )


corpus = []
for i in range(len(list_sentences)):
  review = list_sentences[i].split(' → ')
  review = ' '.join(review)
  corpus.append(review)


# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
cv.fit(corpus)
X = cv.transform(corpus).toarray()
words_df = pd.DataFrame( X )

Data.drop(columns=['Route'],inplace=True)
Data = pd.concat([Data,words_df],axis=1)

In [31]:
list_sentences = []
for id in Test_Data.index.values:
    sentence = Test_Data.at[id,'Route']
    list_sentences.append( sentence )

corpus = []
for i in range(len(list_sentences)):
  review = list_sentences[i].split(' → ')
  review = ' '.join(review)
  corpus.append(review)

X_Test = cv.transform(corpus).toarray()
words_df = pd.DataFrame( X_Test )

Test_Data.drop(columns=['Route'],inplace=True)
Test_Data = pd.concat([Test_Data,words_df],axis=1)

In [32]:
Data.head()

,Dep_Time,Arrival_Time,Duration,Total_Stops,Source_Banglore,Source_Kolkata,Source_Delhi,Source_Chennai,Destination_Delhi,Destination_Banglore,...,32,33,34,35,36,37,38,39,40,41
0,1340,70,170,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,350,795,445,2,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,565,265,1140,2,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1085,1410,325,1,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1010,1295,285,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
Test_Data.head()

,Dep_Time,Arrival_Time,Duration,Total_Stops,Source_Banglore,Source_Kolkata,Source_Delhi,Source_Chennai,Destination_Delhi,Destination_Banglore,...,32,33,34,35,36,37,38,39,40,41
0,1050,265,655,1.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,380,620,240,1.0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,1155,1140,1425,1.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,480,1260,780,1.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1435,165,170,0.0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
# Train Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( Data, target_df, test_size=0.25, random_state=42 )

In [35]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8012, 76), (2671, 76), (8012, 1), (2671, 1))

In [36]:
# ## Box Cox Transformstion
# from sklearn.preprocessing import PowerTransformer
# PT = PowerTransformer( method='yeo-johnson', standardize=True )

# numerical_column_name = ['Duration','Dep_Time','Arrival_Time']
# X_train_try_num_df = X_train[numerical_column_name]
# X_test_try_num_df = X_test[numerical_column_name]

# PT.fit(X_train_try_num_df)
# X_train[numerical_column_name] = PT.transform( X_train_try_num_df )
# X_test[numerical_column_name] = PT.transform( X_test_try_num_df )

In [37]:
# from sklearn.preprocessing import KBinsDiscretizer

# columns_names_temp = [ 'Dep_Time', 'Arrival_Time' ]

# disc = KBinsDiscretizer(n_bins=8, encode='ordinal', strategy='kmeans')
# disc.fit(X_train[columns_names_temp])
# print( disc.bin_edges_ )

# X_train[columns_names_temp] = disc.transform( X_train[columns_names_temp] )
# X_test[columns_names_temp] = disc.transform( X_test[columns_names_temp] )

In [38]:
# Coorealation Matrix
px.imshow( Data[['Duration','Dep_Time','Arrival_Time','Total_Stops']].corr(),color_continuous_scale='RdBu_r', width=900, height=700 )

# Applying ML Model

In [39]:
RMSE_test = []
RMSE_train = []
RMLE_test = []
RMLE_train = []
Model_Name = []

In [40]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor() 
model.fit(X_train, y_train)

y_predictions = model.predict(X_test)
mse = mean_squared_error(y_test, y_predictions)
rmle = mean_squared_log_error( y_test, y_predictions )
rmse = np.sqrt(mse)
score = model.score(X_test,y_test)

RMSE_test.append(rmse)
RMLE_test.append(rmle)
Model_Name.append( 'DecisionTreeRegressor' )

print('Root Mean Square Error : ', rmse )
print('Root Mean Log Error : ', rmle )
print('R2 Score : ', score )

print('\n')

print('Train Set ------------------------')

y_predictions = model.predict(X_train)
mse = mean_squared_error(y_train, y_predictions)
rmle = mean_squared_log_error( y_train, y_predictions )
rmse = np.sqrt(mse)
score = model.score(X_train,y_train)
RMSE_train.append(rmse)
RMLE_train.append(rmle)
print('Root Mean Square Error : ', rmse )
print('Root Mean Log Error : ', rmle )
print('R2 Score : ', score )


Root Mean Square Error :  1523.0848307531012
Root Mean Log Error :  0.022498542981975574
R2 Score :  0.8864373220586687


Train Set ------------------------
Root Mean Square Error :  301.83849433100806
Root Mean Log Error :  0.0010394676742360393
R2 Score :  0.9957705436060569


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names o

In [41]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor() 
model.fit(X_train, y_train)

y_predictions = model.predict(X_test)
mse = mean_squared_error(y_test, y_predictions)
rmle = mean_squared_log_error( y_test, y_predictions )
rmse = np.sqrt(mse)
score = model.score(X_test,y_test)

RMSE_test.append(rmse)
Model_Name.append( 'RandomForestRegressor' )
RMLE_test.append(rmle)


print('Root Mean Square Error : ', rmse )
print('Root Mean Log Error : ', rmle )
print('R2 Score : ', score )

print('\n')

print('Train Set ------------------------')

y_predictions = model.predict(X_train)
mse = mean_squared_error(y_train, y_predictions)
rmle = mean_squared_log_error( y_train, y_predictions )
rmse = np.sqrt(mse)
score = model.score(X_train,y_train)
RMSE_train.append(rmse)
RMLE_train.append(rmle)
print('Root Mean Square Error : ', rmse )
print('Root Mean Log Error : ', rmle )
print('R2 Score : ', score )


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only

Root Mean Square Error :  1462.6360942597853
Root Mean Log Error :  0.015589356477813258
R2 Score :  0.8952726745185619


Train Set ------------------------
Root Mean Square Error :  604.7120524498575
Root Mean Log Error :  0.0028878942038338953
R2 Score :  0.9830241101244603


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.



In [42]:
from xgboost import XGBRegressor
model = XGBRegressor() 
model.fit(X_train, y_train)

y_predictions = model.predict(X_test)
mse = mean_squared_error(y_test, y_predictions)
rmle = mean_squared_log_error( y_test, y_predictions )
rmse = np.sqrt(mse)
score = model.score(X_test,y_test)

RMSE_test.append(rmse)
RMLE_test.append(rmle)
Model_Name.append( 'XGBRegressor' )

print('Root Mean Square Error : ', rmse )
print('Root Mean Log Error : ', rmle )
print('R2 Score : ', score )

print('\n')

print('Train Set ------------------------')

y_predictions = model.predict(X_train)
mse = mean_squared_error(y_train, y_predictions)
rmse = np.sqrt(mse)
rmle = mean_squared_log_error( y_train, y_predictions )
score = model.score(X_train,y_train)
RMSE_train.append(rmse)
RMLE_train.append(rmle)
print('Root Mean Square Error : ', rmse )
print('Root Mean Log Error : ', rmle )
print('R2 Score : ', score )


[23:53:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Root Mean Square Error :  1868.8818303611292
Root Mean Log Error :  0.03662698523276072
R2 Score :  0.8290177124640982


Train Set ------------------------
Root Mean Square Error :  1801.889008887993
Root Mean Log Error :  0.032272154332388114
R2 Score :  0.8492728898911114


In [43]:
# import xgboost as xgb

# params = { 'max_depth': [3,5,10,15,None],
#            'learning_rate': [0.01, 0.05, 0.1],
#            'n_estimators': [100,250,500,1000],
#            'colsample_bytree': [0.3, 0.7]}

# xgbr = xgb.XGBRegressor()
# RS_CV = RandomizedSearchCV(estimator=xgbr, 
#                    param_distributions=params,
#                    scoring='neg_mean_squared_error', 
#                    verbose=3,cv=10)
# RS_CV.fit(X_train, y_train)
# print("Best parameters:", RS_CV.best_params_)
# print("Lowest RMSE: ", (-RS_CV.best_score_)**(1/2.0))

Best parameters: {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 500}
Lowest RMSE:  1468.3565687136681

Best parameters: {'n_estimators': 250, 'max_depth': 10, 'learning_rate': 0.05, 'colsample_bytree': 0.7}
Lowest RMSE:  1438.1784743558496

In [44]:
from xgboost import XGBRegressor
model = XGBRegressor(n_estimators=250,max_depth=4,learning_rate=0.05,colsample_bytree=0.7) 
model.fit(X_train, y_train)

y_predictions = model.predict(X_test)
mse = mean_squared_error(y_test, y_predictions)
rmle = mean_squared_log_error( y_test, y_predictions )
rmse = np.sqrt(mse)
score = model.score(X_test,y_test)

RMSE_test.append(rmse)
RMLE_test.append(rmle)
Model_Name.append( 'XGBRegressor (Tuned)' )

print('Root Mean Square Error : ', rmse )
print('Root Mean Log Error : ', rmle )
print('R2 Score : ', score )

print('\n')

print('Train Set ------------------------')

y_predictions = model.predict(X_train)
mse = mean_squared_error(y_train, y_predictions)
rmse = np.sqrt(mse)
rmle = mean_squared_log_error( y_train, y_predictions )
score = model.score(X_train,y_train)
RMSE_train.append(rmse)
RMLE_train.append(rmle)
print('Root Mean Square Error : ', rmse )
print('Root Mean Log Error : ', rmle )
print('R2 Score : ', score )


[23:53:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Root Mean Square Error :  1683.0439863270194
Root Mean Log Error :  0.02834327364928482
R2 Score :  0.861331322892861


Train Set ------------------------
Root Mean Square Error :  1571.594789838762
Root Mean Log Error :  0.024061178994732195
R2 Score :  0.8853388057488543


In [45]:
# import xgboost as xgb

# params = { 'max_depth': [3,5,10,15],
#            'learning_rate': [0.01, 0.05, 0.1],
#            'n_estimators': [25,50,75,90],
#            'colsample_bytree': [0.3, 0.7]}

# xgbr = xgb.XGBRegressor()
# RS_CV = RandomizedSearchCV(estimator=xgbr, 
#                    param_distributions=params,
#                    scoring='neg_mean_squared_error', 
#                    verbose=3,cv=10)
# RS_CV.fit(X_train, y_train)
# print("Best parameters:", RS_CV.best_params_)
# print("Lowest RMSE: ", (-RS_CV.best_score_)**(1/2.0))

In [46]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor() 
model.fit(X_train, y_train)

y_predictions = model.predict(X_test)
mse = mean_squared_error(y_test, y_predictions)
rmse = np.sqrt(mse)
rmle = mean_squared_log_error( y_test, y_predictions )
score = model.score(X_test,y_test)

RMSE_test.append(rmse)
RMLE_test.append(rmle)
Model_Name.append( 'GradientBoostingRegressor' )

print('Root Mean Square Error : ', rmse )
print('Root Mean Log Error : ', rmle )
print('R2 Score : ', score )

print('\n')

print('Train Set ------------------------')

y_predictions = model.predict(X_train)
mse = mean_squared_error(y_train, y_predictions)
rmle = mean_squared_log_error( y_train, y_predictions )
rmse = np.sqrt(mse)
score = model.score(X_train,y_train)
RMSE_train.append(rmse)
RMLE_train.append(rmle)
print('Root Mean Square Error : ', rmse )
print('Root Mean Log Error : ', rmle )
print('R2 Score : ', score )


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



Root Mean Square Error :  1854.0549403688042
Root Mean Log Error :  0.036207136504975375
R2 Score :  0.8317199477785246


Train Set ------------------------
Root Mean Square Error :  1798.7642970186093
Root Mean Log Error :  0.032133191760680746
R2 Score :  0.849795197778198


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.



In [47]:
# GBR=GradientBoostingRegressor()
# search_grid={'n_estimators':[100,250,500,1000],
#              'learning_rate':[0.01,0.5,0.1],
#              'max_depth':[1,2,3,4,5],
#              'subsample':[.5,.75],
#              'random_state':[42]}

# search=RandomizedSearchCV(estimator=GBR, param_distributions=search_grid, scoring='neg_mean_squared_error',cv=10)
# search.fit(X_train,y_train)
# search.best_params_

{'learning_rate': 0.1,
 'max_depth': 5,
 'n_estimators': 1000,
 'random_state': 42,
 'subsample': 0.5}

In [48]:
from sklearn.linear_model import LinearRegression
model = LinearRegression() 
model.fit(X_train, y_train)

y_predictions = model.predict(X_test)
mse = mean_squared_error(y_test, y_predictions)
rmle = mean_squared_log_error( y_test, y_predictions )
rmse = np.sqrt(mse)
score = model.score(X_test,y_test)

RMSE_test.append(rmse)
RMLE_test.append(rmle)
Model_Name.append( 'LinearRegression' )

print('Root Mean Square Error : ', rmse )
print('Root Mean Log Error : ', rmle )
print('R2 Score : ', score )

print('\n')

print('Train Set ------------------------')

y_predictions = model.predict(X_train)
mse = mean_squared_error(y_train, y_predictions)
rmse = np.sqrt(mse)
rmle = mean_squared_log_error( y_train, y_predictions )
score = model.score(X_train,y_train)
RMSE_train.append(rmse)
RMLE_train.append(rmle)
print('Root Mean Square Error : ', rmse )
print('Root Mean Log Error : ', rmle )
print('R2 Score : ', score )

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning:

Feature names o

Root Mean Square Error :  2392.1610277951145
Root Mean Log Error :  0.06714956582707182
R2 Score :  0.7198644270600002


Train Set ------------------------
Root Mean Square Error :  2420.5249471495
Root Mean Log Error :  0.06285741153161065
R2 Score :  0.7280090673433759


In [49]:
pd.set_option('display.max_rows', None)
results = pd.DataFrame()
results['Model_Name'] = Model_Name
results['RMSE on Test Set'] = RMSE_test
results['RMSE on Train Set'] = RMSE_train
results['RMLE on Test Set'] = RMLE_test
results['RMLE on Train Set'] = RMLE_train
results = results.sort_values(by='RMSE on Test Set')
results

,Model_Name,RMSE on Test Set,RMSE on Train Set,RMLE on Test Set,RMLE on Train Set
1,RandomForestRegressor,1462.636094,604.712052,0.015589,0.002888
0,DecisionTreeRegressor,1523.084831,301.838494,0.022499,0.001039
3,XGBRegressor (Tuned),1683.043986,1571.594790,0.028343,0.024061
4,GradientBoostingRegressor,1854.054940,1798.764297,0.036207,0.032133
2,XGBRegressor,1868.881830,1801.889009,0.036627,0.032272
5,LinearRegression,2392.161028,2420.524947,0.067150,0.062857


# Submission
***

In [50]:
# Final model
final_model = XGBRegressor(n_estimators=250,max_depth=4,learning_rate=0.05,colsample_bytree=0.7)
final_model.fit(X_train,y_train)
final_predictions = final_model.predict(Test_Data)

[23:53:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [51]:
Submission_Data = pd.DataFrame()
Submission_Data['Price'] = final_predictions

In [52]:
# Checking if any SalePrice predicted is negative
Submission_Data[Submission_Data['Price']<=0]

,Price


In [53]:
Submission_Data.head()

,Price
0,14182.973633
1,5130.053711
2,11397.302734
3,10175.675781
4,4143.833984


In [54]:
from google.colab import files
Submission_Data.to_csv('Flight_Tickets_Price_Prediction_Shirsh_Submission_File.csv', encoding = 'utf-8-sig', index=False) 
files.download('Flight_Tickets_Price_Prediction_Shirsh_Submission_File.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>